In [1]:
import pandas as pd
import sqlalchemy as sqla
import pymysql
import requests
import json
import bamboolib as bam

## MySQL connection

In [2]:
user = "gazw6343_ggeoffroy"
password = "CLg#6z[_tUyW"
host = "nout.o2switch.net"
database = "gazw6343_si_publications"

In [3]:
# sqlalchemy
engine = sqla.create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}', pool_recycle=1)

In [61]:
# pymysql
connection = pymysql.connect(host=host,
                             user=user,
                             password=password,
                             db=database)
cursor=connection.cursor()

In [12]:
'''
Create a mapping of df dtypes to mysql data types (not perfect, but close enough)
'''
def dtype_mapping():
    return {'object' : 'TEXT',
        'int64' : 'INT',
         'Int64' : 'INT',
        'float64' : 'FLOAT',
        'datetime64' : 'DATETIME',
        'bool' : 'TINYINT',
        'category' : 'TEXT',
        'timedelta[ns]' : 'TEXT'}

'''
Create sql input for table names and types
'''
def gen_tbl_cols_sql(df):
    dmap = dtype_mapping()
    sql = "id INT AUTO_INCREMENT PRIMARY KEY"
    df1 = df.rename(columns = {"" : "nocolname"})
    hdrs = df1.dtypes.index
    hdrs_list = [(hdr, str(df1[hdr].dtype)) for hdr in hdrs]
    for hl in hdrs_list:
        if hl[0] != 'id':
            sql += " ,{0} {1}".format(hl[0], dmap[hl[1]])
    return sql

'''
Create a mysql table from a df
'''
def create_mysql_tbl_schema(df, connection, db, tbl_name):
    tbl_cols_sql = gen_tbl_cols_sql(df)
    sql = "USE {0}; CREATE TABLE {1} ({2})".format(db, tbl_name, tbl_cols_sql)
    print(sql)
    cur = connection.cursor()
    cur.execute(sql)
    #cur.close()
    #connection.commit()

In [10]:
query = 'SELECT * FROM referentiel_structures LIMIT 5'
df = pd.read_sql_query(sql=sqla.text(query), con=engine.connect())
print(df)

   id  parent_id scopus_id                                              label  \
0   1          0  60110693                             Université Côte d'Azur   
1   2          1  60010513                     Observatoire de la Côte d'Azur   
2   3          1  60122129                                        Villa Arson   
3   4          1  60006380  Institut de Pharmacologie Moléculaire et Cellu...   
4   5          1  60006696  Centre de Recherche sur l'Hétéro-Epitaxie et s...   

         ppn     rnsr_id  hal_id  
0                                 
1  035675683                      
2  02643587X                      
3  05957321X  201220332C  199361  
4  060408278  200017509L     409  


### Reférentiel des structures

In [66]:
df_init = (pd.read_json("../../03_primary/referentiel_structures.json")
.rename(columns={'affiliation_name': 'label','affiliation_id':'scopus_id','ppn_valide': 'ppn','RNSR':'rnsr_id','HAL':'hal_id'})
.drop(['affcourt_valide','documents_count','VIAF', 'ISNI', 'BNF'], axis=1)
.reindex(columns=['id','parent_id','scopus_id','label','ppn', 'rnsr_id', 'hal_id'])
          )

In [ ]:
#genère la requête SQL de création de la table mais avec erreur -> à entre direct dans phpmyAdmin
create_mysql_tbl_schema(df_init, connection, database, "referentiel_strutures")
# pour créer une clé primaire
# ALTER TABLE tbl ADD id INT PRIMARY KEY AUTO_INCREMENT;

In [67]:
df_init.to_sql('referentiel_structures', con = engine, if_exists = 'append', index=False)

50

### référentiel publishers

In [78]:
df = pd.read_csv("../../03_primary/mapping_doiprefixes_publisher.csv", sep=",", encoding="utf-8")
import pandas as pd; import numpy as np
# Step: Change data type of ['prefix', 'publisher_by_doiprefix'] to String/Text
for column_name in df.columns:
    df[column_name] = df[column_name].astype('string')

# Step: Drop missing values in ['publisher_by_doiprefix']
df = df.dropna(subset=['publisher_by_doiprefix'])

# Step: Rename column
df = df.rename(columns={'prefix': 'doi_prefix','publisher_by_doiprefix':'publisher_group'})

df.to_sql('referentiel_publishers', con = engine, if_exists = 'append', index=False)

364

### Publications avec affil

In [133]:
df_struct = pd.read_sql_query(sql=sqla.text("select * from referentiel_structures"), con=engine.connect())

df = pd.read_csv("../../08_reporting/2022-08-29/open_access/publis_all_with_affiliations_data.csv", sep=",", encoding="utf-8")
# Step: Left Join with df_struct where aff_scopus_id=scopus_id
df = pd.merge(df, df_struct[['scopus_id', 'id']], how='left', left_on=['aff_scopus_id'], right_on=['scopus_id'])
import pandas as pd; import numpy as np
# Step: Drop columns
df = df.drop(columns=['scopus_title', 'year', 'creator_author', 'all_authors', 'aff_scopus_id','affiliation_name', 'aff_internal_id', 'aff_parent_id', 'corresponding', 'scopus_id'])

# Step: Rename column
df = df.rename(columns={'source_id': 'scopus_doc_id', 'aff_source_text': 'scopus_source_text', 
                        'author_id': 'scopus_author_id','id': 'affil_id'})

import pandas as pd; import numpy as np
# Step: Change data type of scopus_author_id to String/Text
df['scopus_author_id'] = df['scopus_author_id'].astype('string')

df

               scopus_doc_id                        doi  \
0      SCOPUS_ID:84889561052          10.1111/ger.12094   
1      SCOPUS_ID:84908431549  10.1016/j.lrp.2014.09.001   
2      SCOPUS_ID:84908431549  10.1016/j.lrp.2014.09.001   
3      SCOPUS_ID:84909580664  10.1007/978-1-349-26270-0   
4      SCOPUS_ID:84921605785   10.4324/9781315539171-20   
...                      ...                        ...   
43304  SCOPUS_ID:85135972868    10.1021/acs.cgd.2c00286   
43305  SCOPUS_ID:85136006925       10.1021/jacs.2c05248   
43306  SCOPUS_ID:85136006925       10.1021/jacs.2c05248   
43307  SCOPUS_ID:85136010348       10.1093/imrn/rnab050   
43308  SCOPUS_ID:85136042635   10.1088/2058-9565/ac8444   

                                      scopus_source_text scopus_author_id  \
0      UFR Odontologie, Université Nice Sophia Antipolis      24484677100   
1                    University of Nice-Sophia Antipolis      15757847500   
2                    University of Nice-Sophia Antipolis    

In [ ]:
create_mysql_tbl_schema(df, connection, database, "publis_detail_affil")

In [135]:
df.to_sql('publis_detail_affil', con = engine, if_exists = 'append', index=False)

43309

### Publications uniques metadata

In [4]:
import pandas as pd; import numpy as np
df = pd.read_csv(r'C:\Users\geoffroy\Docker\dagster\app\bso_publis_scopus\08_reporting\2022-08-29\open_access\publis_uniques_doi_oa_data_with_bsoclasses_complete.csv', sep=',', decimal='.', nrows=100000)
# Step: Drop columns
df_publis_metadata = df.drop(columns=['published_date', 'journal_is_oa', 'journal_is_in_doaj', 'oa_status', 'is_oa_normalized', 'oa_status_normalized', 'oa_locations_license', 'oa_locations_host_type', 'oa_host_type_normalized', 'oa_locations_version', 'oa_host_domain', 'oa_hostdomain_count', 'oa_repo_normalized', 'dsm_oa_classification', 'dsm_policy_preprint', 'dsm_romeo_id', 'dsm_splash_url', 'dsm_policy_postprint', 'dsm_policy_published'])

In [5]:
df.shape

(14607, 37)

In [15]:
# Step: Change data type of ['bso_classification_classe', 'is-referenced-by-count'] to Integer
for column_name in ['bso_classification_classe', 'is-referenced-by-count']:
    df_publis_metadata[column_name] = df_publis_metadata[column_name].astype('Int64')

# Step: Rename column
df_publis_metadata = df_publis_metadata.rename(columns={'source_id': 'scopus_doc_id', 'is-referenced-by-count': 'citations'})


df_publis_metadata.columns

Index(['scopus_doc_id', 'doi', 'year', 'corresponding', 'all_authors', 'genre',
       'title', 'publisher', 'journal_name', 'journal_issn_l', 'doi_prefix',
       'publisher_by_doiprefix', 'bso_classification_classe', 'source',
       'bso_classification_en', 'bso_classification_fr', 'citations',
       'funder'],
      dtype='object')

In [ ]:
create_mysql_tbl_schema(df_publis_metadata, connection, database, "publis_metadata")

In [18]:
df_publis_metadata.to_sql('publis_metadata', con = engine, if_exists = 'append', index=False)

14607

### Publis uniques OA data

In [59]:
import pandas as pd; import numpy as np
df = pd.read_csv(r'C:\Users\geoffroy\Docker\dagster\app\bso_publis_scopus\08_reporting\2022-08-29\open_access\publis_uniques_doi_oa_data_with_bsoclasses_complete.csv', sep=',', decimal='.', nrows=100000)
# Step: Drop columns
df = df.drop(columns=['year', 'corresponding', 'all_authors', 'title', 'genre', 'published_date', 'publisher', 'journal_name', 'journal_issn_l', 'oa_status', 'oa_locations_host_type', 'oa_locations_license', 'oa_locations_version', 'oa_host_domain', 'doi_prefix', 'bso_classification_en', 'bso_classification_fr', 'source', 'bso_classification_classe', 'is-referenced-by-count', 'publisher_by_doiprefix', 'dsm_romeo_id', 'dsm_policy_published', 'dsm_policy_postprint', 'dsm_policy_preprint', 'dsm_splash_url', 'dsm_oa_classification', 'funder'])

# Step: Rename column
df = df.rename(columns={'source_id': 'scopus_doc_id'})

# Step: Create new column 'obs_date' from formula '"2022-08-29"'
df['obs_date'] = "2022-08-29"

# Step: Create new column 'obs_date_year' from formula '"2022"'
df['obs_date_year'] = "2022"

df

BamboolibModuleWindow(children=(Window(children=(VBox(children=(VBox(children=(Button(description='Read CSV fi…

In [51]:
create_mysql_tbl_schema(df, connection, database, "publis_oa_data")

USE gazw6343_si_publications; CREATE TABLE publis_oa_data (id INT AUTO_INCREMENT PRIMARY KEY ,scopus_doc_id TEXT ,doi TEXT ,year_upw FLOAT ,journal_is_oa TEXT ,journal_is_in_doaj TEXT ,oa_locations TEXT ,is_oa_normalized TEXT ,oa_status_normalized TEXT ,oa_host_type_normalized TEXT ,oa_hostdomain_count TEXT ,oa_repo_normalized TEXT ,obs_date TEXT ,obs_date_year TEXT)


InterfaceError: (0, '')

In [62]:
df.to_sql('publis_oa_data', con = engine, if_exists = 'append', index=False)

14607